In [287]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib notebook

In [281]:
def generateData(n=10):
    d = np.random.uniform(-1,1,(n,2))
    return d

def targetFunction(random_pts, points, apply=False):
    line_slope = (np.max(random_pts[:,1]) - np.min(random_pts[:,1]))/(np.max(random_pts[:,0]) - np.min(random_pts[:,0]))
    
    if apply:
        points = points.reshape(-1,1)
        x2 = line_slope*(points - np.min(random_pts[:,0])) + np.min(random_pts[:,1])
        data = np.hstack((points, x2))
    else:
        x2 = line_slope*(points[:,0] - np.min(random_pts[:,0])) + np.min(random_pts[:,1])
        yn = np.array(points[:,1] >= x2, dtype=int).reshape(-1,1)
        yn = 2*yn - 1
        data = np.hstack((points, yn))
    return data

def sign(x):
    i = 1 if x > 0 else 0 if x == 0 else -1
    return i

def PLA(x):
    x = np.hstack((np.ones((x.shape[0],1)), x))
    mis_pts = []
    w = np.zeros((1, 3))
    
    choice = np.random.randint(x.shape[0])
    pt = x[choice]
    w[0] = w[0] + pt[3]*pt[:3]
    for p in x:
        if np.int(p[3]) != sign(w[0].dot(p[:3].T)):
            mis_pts.append(p)
    
    it = 1
    
    while mis_pts:
        choice = np.random.randint(len(mis_pts))
        pt = mis_pts[choice]
        w[0] = w[0] + pt[3]*pt[:3]
        mis_pts = []
        for p in x:
            if np.int(p[3]) != sign(w[0].dot(p[:3].T)):
                mis_pts.append(p)
        it += 1
    return (w[0],it)

def run(n, plot=False):
    random_pts = np.random.uniform(-1,1,(2,2))
    line = targetFunction(random_pts, np.linspace(-1,1,n), apply=True)

    data = generateData(n)
    data = targetFunction(random_pts, data)
    color = {-1:'red', 1:'green'}
    p_color = [color[np.int(i)] for i in data[:,-1]]
    w,it = PLA(data)

    if plot:
        plt.title('Target function x Hypothesis')
        plt.plot(line[:,0], line[:,1], label='target function')
        plt.plot(line[:,0], -(w[1]*line_x + w[0])/w[2], label='hypothesis g')
        plt.scatter(data[:,0], data[:,1], c=p_color)
        plt.legend(loc='lower right')
        plt.show()
    return random_pts, w, it

def run_prob(n1, n2):
    random_pts, w, it = run(n1)
    data = generateData(n2)
    x1 = data[:,0]
    x2 = -(w[1]*x1 + w[0])/w[2]

    yn = np.array(data[:,1] >= x2, dtype=int)
    yn = 2*yn - 1

    fx = targetFunction(random_pts, data)[:,-1]
    gx = yn
    return np.sum(fx != gx) / n2

In [258]:
# HW1 Q7
n = 10
it = [run(n)[2] for i in tqdm(range(1000))]
np.mean(it)

10.622

In [285]:
# HW1 Q8
n1 = 10
n2 = 20000

prob = [run_prob(n1,n2) for i in tqdm(range(1000))]
np.mean(prob)

0.18440969999999998

In [289]:
# HW1 Q9
n = 100
it = [run(n)[2] for i in tqdm(range(1000))]
np.mean(it)

100%|██████████| 1000/1000 [00:26<00:00, 37.13it/s]


115.444

In [291]:
# HW1 10
n1 = 100
n2 = 20000

prob = [run_prob(n1,n2) for i in tqdm(range(1000))]
np.mean(prob)

100%|██████████| 1000/1000 [00:29<00:00, 34.24it/s]


0.022872700000000003